# Extract

In [1]:
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

In [ ]:
data = r"C:\Users\bputra\OneDrive - ESRI Indonesia\PT ESRI Indonesia\Research\code-script\17_scrapping_gmaps\gmaps\input\kabkot_list_and_xy_sample.csv"
df = pd.read_csv(data, delimiter=";")
pd.DataFrame(df)

In [ ]:
data_lat = df['lat']
data_long = df['long']

> All Function 

In [5]:
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
def check_exists_by_xpath(xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except:
        return False
    return True


def scroll_to_bottom(iteration=None):
    xpath_sidebar = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    if(iteration == None):
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", xpath_sidebar)
        time.sleep(2)
    else:
        for i in range(1, iteration):
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", xpath_sidebar)
            time.sleep(2)


def scrape_data():
    # Daftar XPath untuk setiap informasi
    xpath_urls = [
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[1]/div[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[3]/div/span[2]/span/span[1]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[2]/span[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[2]/span[2]/span[2]'
    ]
    xpath_href = ['/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a']
    #xpath_href = ['//a[@class="hfpxzc"]']
    
    # List untuk menyimpan semua data
    all_data = []

    # Loop untuk mengekstrak informasi dari setiap elemen
    for i in range(3, 12, 2):  # misalnya, untuk nomor 3, 5, 7, 9, 11
        for xpath in xpath_urls:
            xpath = xpath.format(i)
            #print(xpath)
            try :
                element = driver.find_element(By.XPATH, xpath)
                information = element.text
                all_data.append((information))
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                all_data.append('')
                
        for xpath_xy in xpath_href:
            xpath_xy = xpath_xy.format(i)
            try:
                elements = driver.find_elements(By.XPATH, xpath_xy)
                for element_xy in elements:
                    information_href = element_xy.get_attribute('href')
                    #information_data = element.get_attribute('aria-label')
                    all_data.append((information_href))
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                all_data.append(('',''))

    # Cetak semua data yang telah diambil
    for info in all_data:
        print(info)
            
def url_hit(data_latitude, data_longitude) :
    try : 
        for latitude, longitude in zip (data_latitude, data_longitude) :
            start_time = time.time()
            try : 
                url = f'https://www.google.com/maps/search/Bulutangkis/@{latitude},{longitude},12.58z?entry=ttu' 
                #print(url)
                driver.get(url)
                time.sleep(2)
                
                try :
                    for i in range(1, 100):
                        # scroll down to bottom sidebar
                        scroll_to_bottom()

                        # xpath_endsidebar = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[243]/div/p/span/span')
                        if(check_exists_by_xpath('/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[243]/div/p/span/span')):
                            scroll_to_bottom(2)
                            print("End scroll down !!")
                            
                            #start crawling data !
                            scrape_data()
                            break
                
                except Exception as e :
                    print(f"Ada error ni di request {url} -> {e}")
                    
                print(f'url {url} sudah selesai load')
            
            except TimeoutError :
                print(f'Teeett habis waktu jadi error di {url}')
                
            end_time = time.time()
            
            execution_time = (end_time - start_time)/60
            print(f"Waktu yang dibutuhkan {execution_time:.2f} menit")
            
    except Exception as e :
        print('error nih haduhhh')

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#


### Extract data poi
>> Gambaran Scriptnya 


In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 

# Inisialisasi WebDriver Chrome
driver = webdriver.Chrome()

def scrape_data():
    # Daftar XPath untuk setiap informasi
    xpath_urls = [
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[1]/div[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[3]/div/span[2]/span/span[1]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[2]/span[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[2]/span[2]/span[2]'
    ]
    xpath_href = ['/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a']
    #xpath_href = ['//a[@class="hfpxzc"]']
    
    # List untuk menyimpan semua data
    all_data = []

    # Loop untuk mengekstrak informasi dari setiap elemen
    for i in range(3, 12, 2):  # misalnya, untuk nomor 3, 5, 7, 9, 11
        for xpath in xpath_urls:
            xpath = xpath.format(i)
            #print(xpath)
            try :
                element = driver.find_element(By.XPATH, xpath)
                information = element.text
                all_data.append((information))
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                all_data.append('')
                
        for xpath_xy in xpath_href:
            xpath_xy = xpath_xy.format(i)
            try:
                elements = driver.find_elements(By.XPATH, xpath_xy)
                for element_xy in elements:
                    information_href = element_xy.get_attribute('href')
                    #information_data = element.get_attribute('aria-label')
                    all_data.append((information_href))
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                all_data.append(('',''))

    # Cetak semua data yang telah diambil
    for info in all_data:
        print(info)

# URL target
url = "https://www.google.com/maps/search/bulutangkis/@-6.1712823,106.783929,14z?authuser=0&entry=ttu"
driver.get(url)

# Panggil fungsi scrape_data untuk mengeksekusi scraping
scrape_data()

# Tutup WebDriver setelah selesai
driver.quit()


Diaz Badminton Hall
4,3
Lapangan Bulutangkis
6, Jl. KS Tubun IIC No.29, RT.4/RW.1
0878-8057-5610
https://www.google.com/maps/place/Diaz+Badminton+Hall/data=!4m7!3m6!1s0x2e69f6910123ea9d:0x258d5c895a5c266d!8m2!3d-6.1900129!4d106.8026939!16s%2Fg%2F11b6cls0p1!19sChIJneojAZH2aS4RbSZcWolcjSU?authuser=0&hl=id&rclk=1

Hall Parahyangan Jatibaru
4,1
Lapangan Bulutangkis
Jl. Taman Jati Baru Timur No.24, RT.14/RW.1

https://www.google.com/maps/place/Hall+Parahyangan+Jatibaru/data=!4m7!3m6!1s0x2e69f68157f0066f:0x1202d51218b592f0!8m2!3d-6.1818712!4d106.8126319!16s%2Fg%2F11c2l0jpdj!19sChIJbwbwV4H2aS4R8JK1GBLVAhI?authuser=0&hl=id&rclk=1

Lapangan Bulutangkis PB.KOBSEL
5,0
Lapangan Bulutangkis
Jl.Masjid Al-Hayah No.RT06/04 7, RT.7/RW.4

https://www.google.com/maps/place/Lapangan+Bulutangkis+PB.KOBSEL/data=!4m7!3m6!1s0x2e69f6855f544d6d:0xbc69fe80da23b821!8m2!3d-6.1862713!4d106.8060483!16s%2Fg%2F11dyyz2n75!19sChIJbU1UX4X2aS4RIbgj2oD-abw?authuser=0&hl=id&rclk=1

Lapangan Badminton Tomang
4,6
Lapangan Bu